In [2]:
import sys
import subprocess
import os
import paramiko
import select

repos_folder = os.path.abspath('/mnt/cube/earneodo/repos')
sys.path.append(os.path.join(repos_folder, 'ephysflow'))
sys.path.append(os.path.join(repos_folder, 'ephysflow', 'bci_pipeline'))
from swissknife.bci.core.file import h5_functions as h5f
from basic_viewing.structure import kwik_functions as kwf
from swissknife.bci.core import expstruct as et
from swissknife.bci.core.pipeline.core import kilosort

In [3]:
#logger = logging.getLogger('sort_functions')

def ssh_run(host, cmd):
    #logger.info('Will run ssh command {} on {}'.format(host, cmd))
    ssh = subprocess.Popen(["ssh", "%s" % host, '%s;' % cmd, 'exit;'],
                      shell=False,
                      stdout=subprocess.PIPE,
                      stderr=subprocess.PIPE)

    result = ssh.stdout.readlines()
    if result == []:
        error = ssh.stderr.readlines()
        print >>sys.stderr, "ERROR: %s" % error
        #logger.error('something went wrong')
    else:
        print result
        #logger.info('finished')    
    return result

def ssh_paramiko(host, cmd):
    ssh = paramiko.SSHClient()
    ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())
    ssh.connect(host)
    print "Connected to %s" % host
    stdin, stdout, stderr = ssh.exec_command('py_kilosort.sh {0} {1}'.format(bird, sess))
    msg = []
    while not stdout.channel.exit_status_ready():
        # Only print data if there is data to read in the channel
        if stdout.channel.recv_ready():
            rl, wl, xl = select.select([stdout.channel], [], [], 0.0)
            if len(rl) > 0:
                # Print data from stdout
                print msg.append(stdout.channel.recv(1024))
    print "finished command, returning"
    ssh.close()
    return msg

# Make one single binary file
def make_binary(bird, sess):
    print 'Making binary'
    fn = et.file_names(bird, sess)
    params = et.get_parameters(bird, sess)
    kwd_path = et.file_path(fn, 'ss', 'ss_raw')
    bin_path = et.file_path(fn, 'ss', 'ss_bin')
    chan_list = params['channel_config']['neural']
    h5f.kwd_to_binary(kwd_path, bin_path, chan_list=chan_list)
    print 'done'
#make all the parameters for sorting with kilosort

# sort with a script in niao (scripts gets the data from cube, puts back in cube)
def niao_kilosort(bird, sess):
    print 'Sending sort to niao'
    return ssh_paramiko('niao', 'py_kilosort.sh {0} {1}'.format(bird, sess))

#convert back to kwik file


In [4]:
bird = 'b1114'
sess = 'day-singing_2017-02-23_50'

#print "making binary"
make_binary(bird, sess)
print 'sorting in niao'
niao_msg = niao_kilosort(bird, sess)
print niao_msg
print 'getting back to kwik format'
kwf.kilo_to_kwik(bird, sess)

Making binary


KeyboardInterrupt: 

In [28]:
reload(kwf)

<module 'basic_viewing.kwik_functions' from 'basic_viewing/kwik_functions.pyc'>

In [11]:
kwf.kilo_to_kwik(bird, 'day-morning_2017-02-23_50')